In [1]:
import pandas as pd
import numpy as np
import os
from env import get_db_url
import wrangle
import seaborn as sns
from statistics import stdev
from sklearn.model_selection import train_test_split

In [2]:
def get_zillow_data():
    '''Returns a dataframe of all single family residential properties from 2017.'''
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        sql_query = '''
        SELECT properties_2017.bedroomcnt AS Number_of_Bedrooms,
        properties_2017.bathroomcnt AS Number_of_Bathrooms,
        properties_2017.calculatedfinishedsquarefeet AS Square_Footage, 
        properties_2017.taxvaluedollarcnt AS Tax_Appraised_Value, 
        properties_2017.yearbuilt AS Year_Built, 
        properties_2017.taxamount AS Tax_Assessed, properties_2017.fips AS 'County_Code'
        FROM properties_2017
        JOIN propertylandusetype using (propertylandusetypeid)
        WHERE propertylandusedesc = "Single Family Residential";
        '''
        df = pd.read_sql(sql_query, get_db_url('zillow'))
        df.to_csv(filename)
    return df



In [3]:
df = get_zillow_data()
df.head()

,Number_of_Bedrooms,Number_of_Bathrooms,Square_Footage,Tax_Appraised_Value,Year_Built,Tax_Assessed,County_Code
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [4]:

sql_query = '''
      SELECT properties_2017.bedroomcnt AS Number_of_Bedrooms,
properties_2017.bathroomcnt AS Number_of_Bathrooms,
properties_2017.calculatedfinishedsquarefeet AS Square_Footage, 
properties_2017.taxvaluedollarcnt AS Tax_Appraised_Value, 
properties_2017.yearbuilt AS Year_Built, 
properties_2017.taxamount AS Tax_Assessed, properties_2017.fips AS 'County_Code'
FROM properties_2017
JOIN propertylandusetype using (propertylandusetypeid)
WHERE propertylandusedesc = "Single Family Residential"
       '''
df = pd.read_sql(sql_query, get_db_url('zillow'))

In [5]:
df.head()

,Number_of_Bedrooms,Number_of_Bathrooms,Square_Footage,Tax_Appraised_Value,Year_Built,Tax_Assessed,County_Code
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [6]:
df.shape

(2152863, 7)

In [7]:
df.isnull().mean()

Number_of_Bedrooms     0.000005
Number_of_Bathrooms    0.000005
Square_Footage         0.003941
Tax_Appraised_Value    0.000229
Year_Built             0.004337
Tax_Assessed           0.002063
County_Code            0.000000
dtype: float64

In [8]:
# nulls are a small percentage of the df, good to drop them
df = df.dropna()

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Number_of_Bedrooms,2140235.0,3.301419,0.932600,0.00,3.00,3.00,4.00,25.00
Number_of_Bathrooms,2140235.0,2.240352,0.990549,0.00,2.00,2.00,3.00,32.00
Square_Footage,2140235.0,1863.193970,1221.754161,1.00,1258.00,1623.00,2208.00,952576.00
Tax_Appraised_Value,2140235.0,460641.625164,677157.635675,22.00,189166.00,328296.00,534606.00,90188462.00
Year_Built,2140235.0,1960.967545,22.150563,1801.00,1949.00,1958.00,1976.00,2016.00
Tax_Assessed,2140235.0,5616.711322,7814.562798,6.34,2540.85,4111.47,6411.93,1078101.87
County_Code,2140235.0,6048.309556,20.344910,6037.00,6037.00,6037.00,6059.00,6111.00


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Number_of_Bedrooms   float64
 1   Number_of_Bathrooms  float64
 2   Square_Footage       float64
 3   Tax_Appraised_Value  float64
 4   Year_Built           float64
 5   Tax_Assessed         float64
 6   County_Code          float64
dtypes: float64(7)
memory usage: 130.6 MB


In [32]:
# convert floats to int except taxes and bedrooms
df['County_Code'] = df['County_Code'].astype(int)
df['Year_Built'] = df['Year_Built'].astype(int)
df['Square_Footage'] = df['Square_Footage'].astype(int)
df['Number_of_Bedrooms'] = df['Number_of_Bedrooms'].astype(int)
df['Tax_Appraised_Value'] = df['Tax_Appraised_Value'].astype(int)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2130657 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Number_of_Bedrooms   int64  
 1   Number_of_Bathrooms  float64
 2   Square_Footage       int64  
 3   Tax_Appraised_Value  int64  
 4   Year_Built           int64  
 5   Tax_Assessed         float64
 6   County_Code          int64  
dtypes: float64(2), int64(5)
memory usage: 130.0 MB


In [13]:
# what should the minimum square footage be? 120 
# https://worldpopulationreview.com/state-rankings/minimum-house-size-by-state
# what percentage of homes under 120 sq ft are in the df?
round(len(df[df['Square_Footage']< 120])/ df.shape[0],4)

0.0001

In [14]:
# how many homes have no bedrooms?
len(df[df['Number_of_Bedrooms']==0])

4397

In [15]:
# bathrooms?
len(df[df['Number_of_Bathrooms']==0])

4274

In [16]:
# dropping rows that don't meet minimum home requirements (1 bed .5 bath > 120sq ft)
df = df[df.Square_Footage >120]
df = df[df.Number_of_Bedrooms >0]
df = df[df.Number_of_Bathrooms >0]


In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Number_of_Bedrooms,2134884.0,3.308336,0.921153,1.00,3.00,3.00,4.00,25.00
Number_of_Bathrooms,2134884.0,2.245149,0.985540,0.50,2.00,2.00,3.00,32.00
Square_Footage,2134884.0,1864.197320,1162.389343,127.00,1260.00,1624.00,2209.00,952576.00
Tax_Appraised_Value,2134884.0,460876.590084,674681.418590,22.00,189630.75,328859.00,535000.00,90188462.00
Year_Built,2134884.0,1960.985129,22.133983,1801.00,1949.00,1958.00,1976.00,2016.00
Tax_Assessed,2134884.0,5619.620855,7784.654633,16.48,2546.71,4116.04,6415.76,1078101.87
County_Code,2134884.0,6048.304237,20.324535,6037.00,6037.00,6037.00,6059.00,6111.00


In [18]:
# value counts for each column
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print("-----------------")

Number_of_Bedrooms
3     962631
4     633360
2     334071
5     150594
6      25104
1      22747
7       4791
8       1102
9        290
10       118
11        33
13        15
12        12
14         6
15         5
16         2
18         2
25         1
Name: Number_of_Bedrooms, dtype: int64
-----------------
Number_of_Bathrooms
2.00     942322
3.00     422353
1.00     411769
2.50     142811
4.00      82028
1.50      31155
3.50      28460
5.00      28296
4.50      19469
6.00      10716
5.50       6200
7.00       4380
8.00       1680
6.50       1328
9.00        705
7.50        379
10.00       321
11.00       145
8.50        108
12.00        72
9.50         49
13.00        38
14.00        23
15.00        15
10.50        14
0.50         14
16.00         9
18.00         6
1.75          3
20.00         3
17.00         3
12.50         3
11.50         3
19.50         1
14.50         1
32.00         1
19.00         1
Name: Number_of_Bathrooms, dtype: int64
-----------------
Square_Footage
1200 

In [19]:
# how to handle outliers?
# utilize 7 bed 7.5 bath

df.Number_of_Bathrooms.mean() + (5* stdev(df.Number_of_Bedrooms))

6.850912270417616

In [20]:
df.Number_of_Bathrooms.mean() + (5* stdev(df.Number_of_Bathrooms))

7.172847851521835

In [21]:
df = df[df.Number_of_Bedrooms <=7]
df = df[df.Number_of_Bathrooms <=7.5]

In [22]:
df.Tax_Appraised_Value.mean() + (5* stdev(df.Tax_Appraised_Value))

3330624.634755437

In [23]:
df.Square_Footage.mean() + (10* stdev(df.Square_Footage))

12984.363554740647

In [30]:
df[df.Square_Footage > 13000].mean()

Number_of_Bedrooms     4.894118e+00
Number_of_Bathrooms    5.552941e+00
Square_Footage         1.638096e+04
Tax_Appraised_Value    9.063501e+06
Year_Built             1.984094e+03
Tax_Assessed           1.034852e+05
County_Code            6.046365e+03
dtype: float64

In [25]:
df = df[df.Square_Footage <=56_000]

In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Number_of_Bedrooms,2130657.0,3.301228,0.905162,1.00,3.00,3.00,4.00,7.00
Number_of_Bathrooms,2130657.0,2.233830,0.947761,0.50,2.00,2.00,3.00,7.50
Square_Footage,2130657.0,1850.893710,902.035214,127.00,1259.00,1623.00,2204.00,37634.00
Tax_Appraised_Value,2130657.0,451591.601228,575806.781671,22.00,189416.00,328052.00,533208.00,49386312.00
Year_Built,2130657.0,1960.954404,22.098075,1801.00,1949.00,1958.00,1975.00,2016.00
Tax_Assessed,2130657.0,5511.784768,6605.282642,16.48,2543.86,4109.79,6397.65,560814.33
County_Code,2130657.0,6048.317351,20.333972,6037.00,6037.00,6037.00,6059.00,6111.00


In [27]:
def prepare_zillow_data(df):
    ''' Prepares zillow data'''
    #drop null values
    df = df.dropna()
    #limit homes to 1 bed , .5 bath, and at least 120sq ft     
    df = df[df.Square_Footage > 120]
    df = df[df.Number_of_Bedrooms > 0]
    df = df[df.Number_of_Bathrooms >0]
    # convert floats to int except taxes and bedrooms
    df['County_Code'] = df['County_Code'].astype(int)
    df['Year_Built'] = df['Year_Built'].astype(int)
    df['Square_Footage'] = df['Square_Footage'].astype(int)
    df['Number_of_Bedrooms'] = df['Number_of_Bedrooms'].astype(int)
    # handle outliers homes appraised for less than 3.5M, square footage less than 56,000 and 7 beds and 7.5 baths or less
    df = df[df.Number_of_Bedrooms <=7]
    df = df[df.Number_of_Bathrooms <=7.5]
    df = df[df.Square_Footage <=56_000]
    return df

In [28]:
def split_zillow_data(df):
    ''' This function splits the cleaned dataframe into train, validate, and test 
    datasets and statrifies based on the target - Tax_Appraised_Value.'''

    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)
    
    return train, validate, test

In [29]:
train, validate, test = split_zillow_data(df)